<a href="https://colab.research.google.com/github/isottongloria/Vision-CognitiveSystems/blob/main/MIDI_melody_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 651.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [3]:
import mido
import matplotlib
import numpy as np
import zipfile

## Bytes encoding PENSARCI

In [4]:
n_bars = len(bars)
note_bars_byte = np.zeros((int(n_bars),int(notes_per_bar),8))


for i in range(0,len(bars)):
    bar = bars[i]
    notes_list = []
    for t,j in enumerate(range(1,len(bar)-1,note_length)):
        timestep_bar = bar[j:j+note_length]
        note = most_frequent(timestep_bar)   #most frequent note in the timestep, ignoring silence

        if note!=None :
            note_bars_byte[i,t,:] = np.unpackbits(bytearray(bytes([int(note)])))  #assign one to the corresponding note

NameError: name 'bars' is not defined

In [ ]:
note_bars_byte[0,11,:]

array([0., 1., 0., 0., 1., 0., 1., 1.])

In [ ]:
note_bars_byte.shape

(8, 12, 8)

## Import dataset

In [5]:
zip_file_path = '/content/dataset_lakh.zip'
extract_dir = '/content/'

# Extract all the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

npy_file_path = '/content/dataset_lakh.npy'

# Load the .npy file
loaded_dataset = np.load(npy_file_path)
print(loaded_dataset.shape)

(2698, 8, 64, 128)


# **MODEL**

In [57]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
#import ipdb
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#from ops import *

def conv_prev_concat(x, y):
    """Concatenate conditioning vector on feature map axis."""
    x_shapes = x.shape
    y_shapes = y.shape
    if x_shapes[2:] == y_shapes[2:]:
        y2 = y.expand(x_shapes[0],y_shapes[1],x_shapes[2],x_shapes[3])

        return torch.cat((x, y2),1)

    else:
        print(x_shapes[2:])
        print(y_shapes[2:])


def lrelu(x, leak=0.2):
    z = torch.mul(x,leak)
    return torch.max(x, z)

def batch_norm_1d(x):
    x_shape = x.shape[1]
    batch_nor = nn.BatchNorm1d(x_shape, eps=1e-05, momentum=0.9, affine=True)
    batch_nor = batch_nor.cuda()

    output = batch_nor(x)
    return output

def batch_norm_2d(x):
    x_shape = x.shape[1]
    batch_nor = nn.BatchNorm2d(x_shape, eps=1e-05, momentum=0.9, affine=True)
    batch_nor = batch_nor.cuda()
    output = batch_nor(x)
    return output


class generator(nn.Module):
    def __init__(self,pitch_range):
        super(generator, self).__init__()
        self.gf_dim   = 64
        #self.y_dim   = 13
        self.n_channel = 256

        self.h1      = nn.ConvTranspose2d(in_channels=192, out_channels=pitch_range, kernel_size=(2,1), stride=(2,2))
        self.h2      = nn.ConvTranspose2d(in_channels=192, out_channels=pitch_range, kernel_size=(2,1), stride=(2,2))
        self.h3      = nn.ConvTranspose2d(in_channels=192, out_channels=pitch_range, kernel_size=(2,1), stride=(2,2))
        self.h4      = nn.ConvTranspose2d(in_channels=192, out_channels=1, kernel_size=(1,pitch_range), stride=(1,2))

        self.h0_prev = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(1,pitch_range), stride=(1,2))   # 16 -> 64
        self.h1_prev = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(2,1), stride=(2,2))
        self.h2_prev = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(2,1), stride=(2,2))
        self.h3_prev = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(2,1), stride=(2,2))

        self.linear1 = nn.Linear(100,4096)
        self.linear2 = nn.Linear(4096,self.gf_dim*4*4)

    def forward(self, z, prev_x ,batch_size,pitch_range):

        # h3_prev = F.leaky_relu(self.batch_nor_256(self.h0_prev(prev_x)),0.2)
        h0_prev = lrelu(batch_norm_2d(self.h0_prev(prev_x)),0.2)   #[72, 64, 64, 1]
        print("shapes for the conditioner")
        print(h0_prev.shape)
        h1_prev = lrelu(batch_norm_2d(self.h1_prev(h0_prev)),0.2)  #[72, 64, 32, 1]
        print(h1_prev.shape)
        h2_prev = lrelu(batch_norm_2d(self.h2_prev(h1_prev)),0.2)  #[72, 64, 16, 1]
        print(h2_prev.shape)
        h3_prev = lrelu(batch_norm_2d(self.h3_prev(h2_prev)),0.2)  #[72, 64, 8, 1])
        print(h3_prev.shape)
        #yb = y.view(batch_size,  self.y_dim, 1, 1)  #(72,13,1,1)

        #z = torch.cat((z,y),1)         #(72,113)
        print("shapes for the generator")
        h0 = F.relu(batch_norm_1d(self.linear1(z)))    #(72,4096)
        print(h0.shape)
        #h0 = torch.cat((h0,y),1)   #(72,1037)

        h1 = F.relu(batch_norm_1d(self.linear2(h0)))   #(72, 1024)
        print(h1.shape)
        h1 = h1.view(batch_size, self.gf_dim *2, 2*4, 1)     #(72,128,8,1)
        print(h1.shape)
        #h1 = conv_cond_concat(h1,yb) #(b,141,8,1)
        h1 = conv_prev_concat(h1,h3_prev)  #(72, 144, 8, 1) if y not used,  (72, 157, 8, 1) if y used
        print(h1.shape)

        h2 = F.relu(batch_norm_2d(self.h1(h1)))  #(72, 128, 16, 1)
        print(h2.shape)
        #h2 = conv_cond_concat(h2,yb) #([72, 141, 16, 1])
        h2 = conv_prev_concat(h2,h2_prev)  #([72, 144, 16, 1])
        print(h2.shape)

        h3 = F.relu(batch_norm_2d(self.h2(h2)))  #([72, 128, 32, 1])
        print(h3.shape)
        #h3 = conv_cond_concat(h3,yb)  #([72, 141, 32, 1])
        h3 = conv_prev_concat(h3,h1_prev) #([72, 144, 32, 1])
        print(h3.shape)

        h4 = F.relu(batch_norm_2d(self.h3(h3)))  #([72, 128, 64, 1])
        print(h4.shape)
        #h4 = conv_cond_concat(h4,yb)  #([72, 141, 64, 1])
        h4 = conv_prev_concat(h4,h0_prev) #([72, 144, 64, 1])
        print(h4.shape)

        g_x = torch.sigmoid(self.h4(h4)) #([72, 1, 64, 128])
        print(g_x.shape)
        return g_x


class discriminator(nn.Module):
    def __init__(self,pitch_range):
        super(discriminator, self).__init__()

        self.df_dim = 64
        self.dfc_dim = 1024
        #self.y_dim = 13

        self.h0_prev = nn.Conv2d(in_channels=14, out_channels=14, kernel_size=(2,pitch_range), stride=(2,2))  #ma 14 da dove viene fuori
        #out channels = y_dim +1
        self.h1_prev = nn.Conv2d(in_channels=27, out_channels=77, kernel_size=(4,1), stride=(2,2))
        # out channels = df_dim + y_dim
        self.linear1 = nn.Linear(244,self.dfc_dim)
        self.linear2 = nn.Linear(1037,1)

    def forward(self,x,y,batch_size,pitch_range):

        #yb = y.view(batch_size,self.y_dim, 1, 1)
        #x = conv_cond_concat(x, yb)  #x.shape torch.Size([72, 14, 16, 128])

        h0 = lrelu(self.h0_prev(x),0.2)
        fm = h0
        #h0 = conv_cond_concat(h0, yb) #torch.Size([72, 27, 8, 1])

        h1 = lrelu(batch_norm_2d(self.h1_prev(h0)),0.2)  #torch.Size([72, 77, 3, 1])
        h1 = h1.view(batch_size, -1)  #torch.Size([72, 231])
        #h1 = torch.cat((h1,y),1)  #torch.Size([72, 244])

        h2 = lrelu(batch_norm_1d(self.linear1(h1)))
        #h2 = torch.cat((h2,y),1)  #torch.Size([72, 1037])

        h3 = self.linear2(h2)
        h3_sigmoid = torch.sigmoid(h3)


        return h3_sigmoid, h3, fm

In [7]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        # nn.init.normal_(m.weight.data, 0.0, 0.02)
        nn.init.xavier_uniform_(m.weight.data)
    if classname.find('Linear') != -1:
        #nn.init.normal_(m.weight.data, 0.0, 0.02)
        nn.init.xavier_uniform_(m.weight.data)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.2)
        nn.init.constant_(m.bias.data, 0)

In [8]:
def weights_init_gpt(m):
    classname = m._class.name_
    if classname.find('Conv') != -1:
        # Initialize convolutional layer weights
        nn.init.xavier_uniform_(m.weight.data)  # or use nn.init.normal_ with appropriate parameters
    elif classname.find('Linear') != -1:
        # Initialize linear layer weights
        nn.init.xavier_uniform_(m.weight.data)  # or use nn.init.normal_ with appropriate parameters
    elif classname.find('BatchNorm') != -1:
        # Initialize batch normalization weights (if needed)
        nn.init.normal_(m.weight.data, 1.0, 0.02)  # Example initialization for batch norm

    # Initialize bias terms if they exist
    if hasattr(m, 'bias') and m.bias is not None:
        nn.init.constant_(m.bias.data, 0.0)

In [58]:
device = torch.device('cuda')

noise = torch.randn(72, 100).to(device)

prev_x = loaded_dataset[0:9,:,:,:]
prev_x = torch.from_numpy(prev_x)
prev_x.to(device)
prev_x = prev_x.float()


# Commento di oggi 09/07: credo che sia al contrario, cioè prev_x_res = prev_x.view(72,1,64,128)
# Non l'ho potuto provare perchè colab oggi mi odia a quanto pare
#prev_x_res = prev_x.view(1,72,64,128)
prev_x_res = prev_x.view(72,1,64,128)
prev_x_res = prev_x_res.to(device)

In [59]:
gennaro = generator(128)

In [60]:
gennaro = gennaro.to(device)

In [61]:
generated_bar = gennaro(noise, prev_x_res, 72, 128).detach()
print(generated_bar.shape)

shapes for the conditioner
torch.Size([72, 64, 64, 1])
torch.Size([72, 64, 32, 1])
torch.Size([72, 64, 16, 1])
torch.Size([72, 64, 8, 1])
shapes for the generator
torch.Size([72, 4096])
torch.Size([72, 1024])
torch.Size([72, 128, 8, 1])
torch.Size([72, 192, 8, 1])
torch.Size([72, 128, 16, 1])
torch.Size([72, 192, 16, 1])
torch.Size([72, 128, 32, 1])
torch.Size([72, 192, 32, 1])
torch.Size([72, 128, 64, 1])
torch.Size([72, 192, 64, 1])
torch.Size([72, 1, 64, 128])
torch.Size([72, 1, 64, 128])


In [62]:
generated_bar[0,0,:,:]

tensor([[0.6288, 0.2622, 0.5301,  ..., 0.6883, 0.4885, 0.5026],
        [0.6284, 0.3725, 0.6066,  ..., 0.2318, 0.3651, 0.5578],
        [0.5901, 0.2571, 0.6481,  ..., 0.4928, 0.3160, 0.6349],
        ...,
        [0.6638, 0.4620, 0.5230,  ..., 0.4737, 0.4403, 0.5013],
        [0.6516, 0.4310, 0.7133,  ..., 0.4965, 0.6449, 0.4530],
        [0.6170, 0.2232, 0.5867,  ..., 0.4387, 0.4986, 0.4768]],
       device='cuda:0')

In [ ]:
prev_x_res.shape

torch.Size([1, 72, 64, 128])

In [ ]:
print(gennaro)

generator(
  (h1): ConvTranspose2d(144, 128, kernel_size=(2, 1), stride=(2, 2))
  (h2): ConvTranspose2d(144, 128, kernel_size=(2, 1), stride=(2, 2))
  (h3): ConvTranspose2d(144, 128, kernel_size=(2, 1), stride=(2, 2))
  (h4): ConvTranspose2d(144, 1, kernel_size=(1, 128), stride=(1, 2))
  (h0_prev): Conv2d(1, 64, kernel_size=(1, 128), stride=(1, 2))
  (h1_prev): Conv2d(64, 64, kernel_size=(2, 1), stride=(2, 2))
  (h2_prev): Conv2d(64, 64, kernel_size=(2, 1), stride=(2, 2))
  (h3_prev): Conv2d(64, 64, kernel_size=(2, 1), stride=(2, 2))
  (linear1): Linear(in_features=100, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=256, bias=True)
)
